In [1]:
import random
import os
from glob import glob
import pandas as pd

from info import pj

In [2]:
TEMPERATURE = 0.1
MAX_OUTPUT_TOKENS = 1024
TOP_P = 0.95
TOP_K = 40

In [3]:
composer = "Wolfgang Amadeus Mozart"
piece_title = "Overture to the Magic Flute"

# No context generation using PaLM

In [4]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project = pj, location="us-central1")

In [5]:
parameters = {
    "temperature": TEMPERATURE,
    "max_output_tokens": MAX_OUTPUT_TOKENS,
    "top_p": TOP_P,
    "top_k": TOP_K
}
model = TextGenerationModel.from_pretrained("text-bison@001")

response = model.predict(
    """\
    Write a classical music program notes for tonight\'s performance of {} by {}.\
    Incorporate relevant historical events at the time of this piece\'s composition.\
    Also incorporate relevant events from the composer\'s life at the time of this piece\'s composition.\
    """.format(piece_title, composer),
    **parameters
)

print(response.text)

The Overture to The Magic Flute is one of the most famous and beloved pieces of classical music ever written. It was composed by Wolfgang Amadeus Mozart in 1791, and it premiered in Vienna on September 30 of that year. The opera itself is a Singspiel, or a German opera that includes spoken dialogue. The story is based on a fairy tale by Emanuel Schikaneder, and it tells the story of a young prince named Tamino who is on a quest to rescue a beautiful princess named Pamina from the evil sorcerer Sarastro.

The Overture to The Magic Flute is a brilliant piece of music that perfectly captures the excitement and drama of the opera. It begins with a slow, mysterious introduction that sets the scene for the story. The music then builds in intensity as Tamino embarks on his quest, and it reaches a climax in the final section, when Tamino and Pamina are reunited.

The Overture to The Magic Flute was composed at a time when Mozart was at the height of his powers. He had already written some of h

# Adding Context to prompt using Langchain

In [6]:
from langchain.document_loaders import TextLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import SKLearnVectorStore
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

In [7]:
loaded_docs = []
for file in glob("./example_notes/*mozart*.txt"):
    try:
        loader = TextLoader(file, encoding="utf8")
        loaded_docs.extend(loader.load())
    except Exception as e:
        loader = TextLoader(file, autodetect_encoding=True)
        loaded_docs.extend(loader.load())

# text_splitter = CharacterTextSplitter(separator = "\n")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
docs = text_splitter.split_documents(loaded_docs)

In [8]:
embeddings = VertexAIEmbeddings()

In [15]:
custom_template = """\
Write a classical music program notes for tonight\'s performance of {piece_title} by {composer}\
Incorporate relevant historical events at the time of this piece\'s composition.\
Also incorporate relevant events from the composer\'s life at the time of this piece\'s composition.\
Incorporate information from the Context section.\

Context:
{context}
"""

## Pull up context

In [10]:
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embeddings
)

In [16]:
context_query = "Retrieve relevant text for {} by {}".format(piece_title, composer)

addl_context = vector_store.similarity_search(query = context_query, k = 7) 
addl_context

[Document(page_content='wolfgang amadeus mozart: Overture to The Magic Flute, K. 620\nMozart’s incomparable musical gifts enabled him to compose at the highest level of artistic brilliance in almost every musical genre. We are privileged to experience his legacy in symphonies, chamber music, wind serenades, choral music, keyboard music—the list goes on and on. But, unquestionably, his greatest contributions to musical art are his operas. No one—not even Wagner, Verdi, Puccini, or Richard Strauss excelled the perfection of Mozart’s mature operas. The reason, of course, is clear: his unparalleled musical gift is served and informed by a nuanced insight into human psychology that is simply stunning. While Mozart composed both comic operas and serious operas, and in both German and Italian, his major body of work lies in his opera buffe--Italian comic operas. Almost every music lover cherishes his Cosi fan tutte, The Marriage of Figaro, and Don Giovanni, but his last opera is rather differ

In [17]:
prompt = PromptTemplate(template = custom_template,
                        input_variables = ["context", "composer", "piece_title"])
prompt_text = prompt.format(context = addl_context,
                            composer = composer,
                            piece_title = piece_title)

In [18]:
llm = VertexAI(model_name = "text-bison@001",
               temperature = TEMPERATURE,
               max_output_tokens = MAX_OUTPUT_TOKENS,
               top_p = TOP_P,
               top_k = TOP_K)

In [19]:
vectored_output = llm(prompt_text)

print(vectored_output)

Wolfgang Amadeus Mozart's Overture to The Magic Flute is a masterpiece of musical theater. It was composed in 1791, the same year that Mozart died. The opera itself is a singspiel, which is a type of opera that combines spoken dialogue with musical numbers. The Magic Flute is a curious blend of the sublime and the silly, incorporating oblique references to rituals of the Masons. It was a considerable success and might have been the start of a whole new career for Mozart, had he not taken ill and died in the middle of its run.

The overture is a fugal sonata movement of the sort that Mozart had composed for the finale of his "Jupiter" Symphony three years earlier. The impressive chords that open the overture (and return between the exposition and the development) are taken from Act I of the opera, where they signify the solemnity of the sacred temple. The tugging impulsiveness of the fugue subject (which is not taken from the opera) is caused by displaced accents: Mozart marks the downb

## Evaluate

In [20]:
from langchain.chains import RetrievalQA
from langchain.evaluation.qa import QAGenerateChain
import langchain
langchain.debug = False

In [21]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type = "stuff",
    retriever = vector_store.as_retriever(),
    verbose = True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [39]:
examples = [
    {
        "query": "What key is the Overture to the Magic Flute in?",
        "answer": "E flat major"
    },
    {
        "query": "Does the Magic Flute contain Freemasonic symbolism?",
        "answer": "Yes"
    }
]

In [40]:
example_gen_chain = QAGenerateChain.from_llm(llm)

new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in random.sample(addl_context, 3)]
)

examples += new_examples

In [33]:
from langchain.evaluation.qa import QAEvalChain
import langchain
langchain.debug = False

In [41]:
predictions = qa.apply(examples)



> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


In [42]:
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(examples, predictions)

In [43]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: What key is the Overture to the Magic Flute in?
Real Answer: E flat major
Predicted Answer: The Overture to the Magic Flute is in the key of Eb.
Predicted Grade: CORRECT

Example 1:
Question: Does the Magic Flute contain Freemasonic symbolism?
Real Answer: Yes
Predicted Answer: Yes, The Magic Flute contains Freemasonic symbolism.
Predicted Grade: CORRECT

Example 2:
Question: What was the name of the singer-actor-writer-promoter-manager-theater owner who collaborated with Mozart on The Magic Flute?
Real Answer: Emanuel Schikaneder
Predicted Answer: The singer-actor-writer-promoter-manager-theater owner who collaborated with Mozart on The Magic Flute was Emanuel Schikaneder.
Predicted Grade: CORRECT

Example 3:
Question: What is the name of the movement that Mozart composed for the finale of his "Jupiter" Symphony?
Real Answer: fugal sonata movement
Predicted Answer: The last movement is one that features a jolly theme that returns frequently after being intersperse